In [1]:
import random
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from copy import deepcopy
import data_generation as dg
import time

In [2]:
graph, random_solution, total_edge_length = dg.read_instance('data\data70.txt')
num_nodes = len(graph)

In [3]:
sols = []
values = []
iters = []
times = []

<span style="font-size: larger;">Classic local search</span>

In [4]:
def make_change_swap(graph, solution):
    new_solution = solution.copy()
    
    # Choose two distinct random indices
    index1, index2 = random.sample(range(len(solution)), 2)
    
    # Swap the positions of the selected nodes
    new_solution[index1], new_solution[index2] = new_solution[index2], new_solution[index1]

    return new_solution

In [5]:
def make_change_inverse(graph, solution):
    new_solution = solution.copy()
    
    # Choose two distinct random indices
    index1, index2 = sorted(random.sample(range(len(solution)), 2))
    
    # Reverse the subset of nodes between index1 and index2
    new_solution[index1:index2+1] = reversed(new_solution[index1:index2+1])

    return new_solution

In [6]:
def make_change_next_permutation(graph, solution):
    #based on the classic next permutation algorithm
    new_solution = deepcopy(solution)
    
    n = len(new_solution)
    
    # Find the largest index k such that a[k] < a[k+1]
    k = n - 2
    while k >= 0 and new_solution[k] >= new_solution[k + 1]:
        k -= 1

    # If no such index exists, the permutation is the last one
    if k == -1:
        return sorted(new_solution)

    # Find the largest index l greater than k such that a[k] < a[l]
    l = n - 1
    while new_solution[k] >= new_solution[l]:
        l -= 1

    # Swap a[k] and a[l]
    new_solution[k], new_solution[l] = new_solution[l], new_solution[k]

    # Reverse the sequence from a[k+1] up to and including the final element a[n-1]
    new_solution[k + 1:] = reversed(new_solution[k + 1:])

    return new_solution


In [7]:
def make_change_scramble(graph, solution):
    new_solution = deepcopy(solution)
    
    start, end = sorted(random.sample(range(len(solution)), 2))

    subset = solution[start:end+1]

    random.shuffle(subset)

    new_solution[start:end+1] = subset

    return new_solution

In [8]:
def local_search(graph, random_solution, value, num_iters, change_func):
    solution = deepcopy(random_solution)
    best_solution = deepcopy(solution)
    best_value = value
    best_i = None

    for i in range(num_iters):
        #print(solution, value, i)
        new_solution = change_func(graph, solution)
        new_value = dg.calculate_total_edge_length(graph, new_solution)

        if new_value < value:
            value = new_value
            solution = deepcopy(new_solution)

            if new_value < best_value:
                best_i = i
                best_value = new_value
                best_solution = deepcopy(new_solution)

    return best_solution, best_value, best_i

In [9]:
start = time.time()
solution, value, iter = local_search(graph, random_solution, total_edge_length, 500, change_func=make_change_swap)
end = time.time()
duration = float("{:.2f}".format(end - start))
print(solution, value, iter)
sols.append(solution)
values.append(value)
iters.append(iter)
times.append(duration)

[14, 55, 20, 22, 1, 67, 7, 64, 6, 19, 13, 68, 10, 58, 53, 63, 17, 31, 51, 27, 41, 65, 57, 40, 44, 32, 21, 5, 11, 47, 29, 60, 69, 59, 61, 8, 0, 35, 16, 12, 66, 2, 48, 56, 34, 52, 37, 46, 15, 36, 54, 18, 33, 38, 25, 30, 4, 9, 43, 26, 28, 62, 23, 42, 45, 39, 50, 24, 49, 3] 24923 483


In [10]:
start = time.time()
solution, value, iter = local_search(graph, random_solution, total_edge_length, 500, change_func=make_change_inverse)
end = time.time()
duration = float("{:.2f}".format(end - start))
print(solution, value, iter)
sols.append(solution)
values.append(value)
iters.append(iter)
times.append(duration)

[1, 55, 20, 10, 3, 30, 22, 57, 9, 68, 33, 29, 32, 27, 14, 53, 67, 44, 42, 13, 31, 36, 18, 35, 21, 26, 54, 40, 49, 41, 51, 46, 17, 69, 63, 2, 37, 64, 4, 6, 12, 15, 7, 24, 39, 66, 60, 11, 65, 0, 47, 5, 59, 19, 48, 34, 61, 56, 52, 8, 50, 62, 16, 23, 45, 58, 28, 25, 43, 38] 25783 495


In [11]:
start = time.time()
solution, value, iter = local_search(graph, random_solution, total_edge_length, 500, change_func=make_change_scramble)
end = time.time()
duration = float("{:.2f}".format(end - start))
print(solution, value, iter)
sols.append(solution)
values.append(value)
iters.append(iter)
times.append(duration)

[30, 28, 56, 43, 41, 23, 45, 57, 58, 54, 1, 39, 62, 34, 7, 47, 22, 61, 59, 38, 32, 11, 46, 6, 69, 5, 8, 21, 51, 12, 35, 36, 16, 63, 2, 52, 4, 29, 55, 24, 26, 42, 67, 48, 33, 19, 44, 53, 65, 0, 13, 66, 18, 60, 25, 40, 9, 31, 64, 27, 10, 15, 37, 68, 20, 17, 50, 49, 14, 3] 25560 498


In [12]:
#generating comparison table
from IPython.display import display
import pandas as pd
dim = num_nodes

results = []

best = float('inf')
best_i = -1

average = [0,0,0]

number_of_combinations = 0

solution_generation = [make_change_swap, make_change_inverse, make_change_scramble]
for i, sg in enumerate(solution_generation):
    method = sg.__name__
    results.append({'Dim': dim, 'Method': method, 'Value': values[i], 'Time': times[i], 'Best_Iter': iters[i]})

    if values[i] == best:
        if times[i] <= times[best_i]:
            best = values[i]
            best_i = i

    if values[i] < best:
        best = values[i]
        best_i = i

    average[0] += values[i]
    average[1] += times[i] 
    average[2] += iters[i]

    number_of_combinations += 1

df = pd.DataFrame(results)
#display(df)
display(df.drop('Best_Iter', axis=1)) 

,Dim,Method,Value,Time
0,70,make_change_swap,24923,0.97
1,70,make_change_inverse,25783,0.96
2,70,make_change_scramble,25560,0.85


In [13]:
print('best:', solution_generation[best_i].__name__, values[best_i], times[best_i], iters[best_i])
df_best = pd.DataFrame({'Dim': dim, 'Method':  solution_generation[best_i].__name__, 'Value': values[best_i], 'Time': times[best_i], 'Best_Iter': iters[best_i]}, index=[0])
display(df_best)

best: make_change_swap 24923 0.97 483


,Dim,Method,Value,Time,Best_Iter
0,70,make_change_swap,24923,0.97,483


In [14]:
df = pd.read_csv('comparison_tables/bests.csv')
row_to_update = df[df['Dim'] == num_nodes]

# Check if the row exists
if not row_to_update.empty:
    # Update specific columns in the located row
    df.loc[row_to_update.index, 'local_search'] = values[best_i]
    df.loc[row_to_update.index, 'local_search_time'] = times[best_i]

    # Save the updated DataFrame back to the CSV file
    df.to_csv('comparison_tables/bests.csv', index=False)
else:
    new_row_data = {'Dim': num_nodes, 'local_search': value, 'local_search_time': duration}
    df.loc[len(df)] = new_row_data
    df.to_csv('comparison_tables/bests.csv', index=False)

In [15]:
average = [average[0] / number_of_combinations, average[1] / number_of_combinations, average[2] / number_of_combinations]
average = [round(num, 2) for num in average]
print('average:', average)
df_avg = pd.DataFrame({'Dim': dim, 'Value': average[0], 'Time': average[1], 'Best_Iter': average[2]}, index=[0])
display(df_avg)

average: [25422.0, 0.93, 492.0]


,Dim,Value,Time,Best_Iter
0,70,25422.0,0.93,492.0


In [16]:
df = pd.read_csv('comparison_tables/averages.csv')
row_to_update = df[df['Dim'] == num_nodes]

# Check if the row exists
if not row_to_update.empty:
    # Update specific columns in the located row
    df.loc[row_to_update.index, 'local_search'] = average[0]
    df.loc[row_to_update.index, 'local_search_time'] = average[1]

    # Save the updated DataFrame back to the CSV file
    df.to_csv('comparison_tables/averages.csv', index=False)
else:
    df.loc[len(df)] = {'Dim': num_nodes, 'local_search': average[0], 'local_search_time': average[1]}
    df.to_csv('comparison_tables/averages.csv', index=False)

<span style="font-size: larger;">Local search with next permutation help</span>

In [17]:
sols = []
values = []
iters = []
times = []

def local_search_with_permutation(graph, random_solution, value, num_iters, change_func):
    solution = deepcopy(random_solution)
    best_solution = deepcopy(solution)
    best_value = value
    best_i = None

    for i in range(num_iters):
        #print(solution, value, i)
        new_solution = change_func(graph, solution)
        new_value = dg.calculate_total_edge_length(graph, new_solution)

        if new_value < value:
            value = new_value
            solution = deepcopy(new_solution)

            if new_value < best_value:
                best_i = i
                best_value = new_value
                best_solution = deepcopy(new_solution)

        else:
            perm_counter = 0
            perm_limit = num_iters / (i+1)
            while new_value >= value and perm_counter < perm_limit:    
                new_solution = make_change_next_permutation(graph, new_solution)
                new_value = dg.calculate_total_edge_length(graph, new_solution)
                perm_counter += 1

            if new_value < value:
                value = new_value
                solution = deepcopy(new_solution)

                if new_value < best_value:
                    best_i = i
                    best_value = new_value
                    best_solution = deepcopy(new_solution)
              

    return best_solution, best_value, best_i

In [18]:
start = time.time()
solution, value, iter = local_search_with_permutation(graph, random_solution, total_edge_length, 500, change_func=make_change_inverse)
end = time.time()
duration = float("{:.2f}".format(end - start))
print(solution, value, iter)
sols.append(solution)
values.append(value)
iters.append(iter)
times.append(duration)

[47, 12, 43, 58, 23, 9, 49, 38, 63, 46, 28, 34, 27, 62, 30, 56, 59, 18, 32, 50, 21, 39, 35, 0, 20, 16, 45, 60, 42, 44, 31, 26, 11, 33, 17, 64, 25, 36, 10, 8, 13, 48, 29, 19, 69, 53, 54, 3, 57, 4, 68, 2, 52, 14, 51, 15, 7, 37, 24, 66, 6, 40, 5, 65, 67, 55, 1, 61, 22, 41] 25982 497


In [19]:
start = time.time()
solution, value, iter = local_search_with_permutation(graph, random_solution, total_edge_length, 500, change_func=make_change_swap)
end = time.time()
duration = float("{:.2f}".format(end - start))
print(solution, value, iter)
sols.append(solution)
values.append(value)
iters.append(iter)
times.append(duration)

[10, 56, 45, 57, 43, 1, 7, 6, 55, 15, 14, 46, 34, 61, 4, 12, 39, 5, 51, 47, 58, 32, 23, 41, 67, 30, 63, 11, 38, 59, 49, 53, 17, 9, 8, 13, 29, 36, 64, 60, 19, 27, 33, 21, 16, 31, 28, 26, 35, 18, 62, 65, 44, 42, 54, 22, 0, 2, 66, 52, 68, 69, 48, 25, 50, 24, 37, 3, 40, 20] 25261 486


In [20]:
start = time.time()
solution, value, iter = local_search_with_permutation(graph, random_solution, total_edge_length, 500, change_func=make_change_scramble)
end = time.time()
duration = float("{:.2f}".format(end - start))
print(solution, value, iter)
sols.append(solution)
values.append(value)
iters.append(iter)
times.append(duration)

[6, 14, 62, 22, 30, 20, 4, 55, 3, 17, 19, 42, 51, 29, 44, 27, 18, 9, 35, 64, 63, 31, 32, 53, 7, 50, 11, 54, 16, 40, 69, 26, 56, 8, 39, 59, 47, 43, 46, 52, 21, 38, 60, 25, 48, 65, 36, 49, 13, 2, 41, 61, 57, 24, 34, 15, 12, 45, 33, 58, 0, 66, 67, 23, 37, 5, 1, 68, 10, 28] 25741 498


In [21]:
#generating comparison table
from IPython.display import display
import pandas as pd
dim = num_nodes

results = []

best = float('inf')
best_i = -1

average = [0,0,0]

number_of_combinations = 0

solution_generation = [make_change_swap, make_change_inverse, make_change_scramble]
for i, sg in enumerate(solution_generation):
    method = sg.__name__ + '_with_permutation'
    results.append({'Dim': dim, 'Method': method, 'Value': values[i], 'Time': times[i], 'Best_Iter': iters[i]})

    if values[i] == best:
        if times[i] <= times[best_i]:
            best = values[i]
            best_i = i

    if values[i] < best:
        best = values[i]
        best_i = i    

    average[0] += values[i]
    average[1] += times[i] 
    average[2] += iters[i]

    number_of_combinations += 1

df = pd.DataFrame(results)
#display(df)
display(df.drop('Best_Iter', axis=1)) 

,Dim,Method,Value,Time
0,70,make_change_swap_with_permutation,25982,3.90
1,70,make_change_inverse_with_permutation,25261,5.99
2,70,make_change_scramble_with_permutation,25741,6.83


In [22]:
print('best:', solution_generation[best_i].__name__, values[best_i], times[best_i], iters[best_i])
df_best = pd.DataFrame({'Dim': dim, 'Method':  solution_generation[best_i].__name__ + '_with_permutation', 'Value': values[best_i], 'Time': times[best_i], 'Best_Iter': iters[best_i]}, index=[0])
display(df_best)

best: make_change_inverse 25261 5.99 486


,Dim,Method,Value,Time,Best_Iter
0,70,make_change_inverse_with_permutation,25261,5.99,486


In [23]:
df = pd.read_csv('comparison_tables/bests.csv')
row_to_update = df[df['Dim'] == num_nodes]

# Check if the row exists
if not row_to_update.empty:
    # Update specific columns in the located row
    df.loc[row_to_update.index, 'local_search_perm'] = values[best_i]
    df.loc[row_to_update.index, 'local_search_perm_time'] = times[best_i]

    # Save the updated DataFrame back to the CSV file
    df.to_csv('comparison_tables/bests.csv', index=False)
else:
    new_row_data = {'Dim': num_nodes, 'local_search_perm': value, 'local_search_perm_time': duration}
    df.loc[len(df)] = new_row_data
    df.to_csv('comparison_tables/bests.csv', index=False)

In [24]:
average = [average[0] / number_of_combinations, average[1] / number_of_combinations, average[2] / number_of_combinations]
average = [round(num, 2) for num in average]
print('average:', average)
df_avg = pd.DataFrame({'Dim': dim, 'Value': average[0], 'Time': average[1], 'Best_Iter': average[2]}, index=[0])
display(df_avg)

average: [25661.33, 5.57, 493.67]


,Dim,Value,Time,Best_Iter
0,70,25661.33,5.57,493.67


In [25]:
df = pd.read_csv('comparison_tables/averages.csv')
row_to_update = df[df['Dim'] == num_nodes]

# Check if the row exists
if not row_to_update.empty:
    # Update specific columns in the located row
    df.loc[row_to_update.index, 'local_search_perm'] = average[0]
    df.loc[row_to_update.index, 'local_search_perm_time'] = average[1]

    # Save the updated DataFrame back to the CSV file
    df.to_csv('comparison_tables/averages.csv', index=False)
else:
    df.loc[len(df)] = {'Dim': num_nodes, 'local_search_perm': average[0], 'local_search_perm_time': average[1]}
    df.to_csv('comparison_tables/averages.csv', index=False)

<span style="font-size: larger;">Simulated annealing</span>

In [26]:
sols = []
values = []
iters = []
times = []

def simulated_annealing(graph, random_solution, value, num_iters, change_func):
    solution = deepcopy(random_solution)
    best_solution = deepcopy(solution)
    best_value = value
    best_i = None

    for i in range(1, num_iters + 1):
        #print(solution, value)
        new_solution = change_func(graph, solution)
        new_value = dg.calculate_total_edge_length(graph, new_solution)

        if new_value < value:
            value = new_value
            solution = deepcopy(new_solution)

            if new_value < best_value:
                best_i = i
                best_value = new_value
                best_solution = deepcopy(new_solution)

        #elif random.random() < 1 / (i**0.5):
        elif random.random() < 1 / i:
            #print('divs')
            value = new_value
            solution = deepcopy(new_solution)

    return best_solution, best_value, best_i

In [27]:
start = time.time()
solution, value, iter = simulated_annealing(graph, random_solution, total_edge_length, 500, change_func=make_change_swap)
end = time.time()
duration = float("{:.2f}".format(end - start))
print(solution, value, iter)
sols.append(solution)
values.append(value)
iters.append(iter)
times.append(duration)

[9, 39, 50, 38, 7, 25, 3, 49, 24, 1, 30, 17, 37, 14, 40, 6, 12, 41, 51, 42, 46, 23, 60, 66, 67, 47, 65, 20, 63, 27, 69, 15, 61, 22, 11, 36, 13, 29, 53, 59, 58, 26, 64, 57, 68, 2, 31, 10, 21, 18, 52, 8, 48, 16, 0, 43, 32, 54, 34, 5, 56, 35, 62, 44, 45, 33, 28, 55, 4, 19] 25119 497


In [28]:
start = time.time()
solution, value, iter = simulated_annealing(graph, random_solution, total_edge_length, 500, change_func=make_change_inverse)
end = time.time()
duration = float("{:.2f}".format(end - start))
print(solution, value, iter)
sols.append(solution)
values.append(value)
iters.append(iter)
times.append(duration)

[37, 49, 43, 28, 26, 69, 50, 67, 40, 6, 36, 22, 65, 33, 52, 64, 24, 48, 3, 5, 42, 66, 15, 2, 7, 55, 60, 51, 25, 27, 17, 14, 53, 46, 61, 45, 16, 31, 4, 30, 35, 63, 29, 32, 57, 11, 8, 56, 38, 47, 10, 54, 21, 34, 59, 44, 39, 58, 13, 0, 19, 18, 12, 23, 62, 1, 20, 41, 9, 68] 26105 495


In [29]:
start = time.time()
solution, value, iter = simulated_annealing(graph, random_solution, total_edge_length, 500, change_func=make_change_scramble)
end = time.time()
duration = float("{:.2f}".format(end - start))
print(solution, value, iter)
sols.append(solution)
values.append(value)
iters.append(iter)
times.append(duration)

[25, 7, 1, 6, 66, 14, 3, 67, 57, 41, 51, 55, 29, 22, 19, 13, 64, 37, 59, 69, 12, 20, 23, 50, 58, 2, 36, 40, 53, 0, 38, 63, 60, 61, 32, 11, 35, 47, 27, 31, 65, 52, 21, 10, 17, 9, 30, 44, 46, 33, 34, 54, 48, 26, 8, 56, 39, 43, 5, 68, 16, 18, 15, 42, 49, 45, 4, 62, 28, 24] 25241 488


In [30]:
#generating comparison table
from IPython.display import display
import pandas as pd
dim = num_nodes

results = []

best = float('inf')
best_i = -1

average = [0,0,0]

number_of_combinations = 0

solution_generation = [make_change_swap, make_change_inverse, make_change_scramble]
for i, sg in enumerate(solution_generation):
    method = sg.__name__
    results.append({'Dim': dim, 'Method': 'simulated_annealing' + ', '  + method, 'Value': values[i], 'Time': times[i], 'Best_Iter': iters[i]})

    if values[i] == best:
        if times[i] <= times[best_i]:
            best = values[i]
            best_i = i

    if values[i] < best:
        best = values[i]
        best_i = i

    average[0] += values[i]
    average[1] += times[i] 
    average[2] += iters[i]

    number_of_combinations += 1

df = pd.DataFrame(results)
#display(df)
display(df.drop('Best_Iter', axis=1)) 

,Dim,Method,Value,Time
0,70,"simulated_annealing, make_change_swap",25119,1.14
1,70,"simulated_annealing, make_change_inverse",26105,0.96
2,70,"simulated_annealing, make_change_scramble",25241,1.01


In [31]:
print('best:', solution_generation[best_i].__name__, values[best_i], times[best_i], iters[best_i])
df_best = pd.DataFrame({'Dim': dim, 'Method':  'simulated_annealing' + ', '  + solution_generation[best_i].__name__, 'Value': values[best_i], 'Time': times[best_i], 'Best_Iter': iters[best_i]}, index=[0])
display(df_best)

best: make_change_swap 25119 1.14 497


,Dim,Method,Value,Time,Best_Iter
0,70,"simulated_annealing, make_change_swap",25119,1.14,497


In [32]:
df = pd.read_csv('comparison_tables/bests.csv')
row_to_update = df[df['Dim'] == num_nodes]

# Check if the row exists
if not row_to_update.empty:
    # Update specific columns in the located row
    df.loc[row_to_update.index, 'sim_annealing'] = values[best_i]
    df.loc[row_to_update.index, 'sim_annealing_time'] = times[best_i]

    # Save the updated DataFrame back to the CSV file
    df.to_csv('comparison_tables/bests.csv', index=False)
else:
    new_row_data = {'Dim': num_nodes, 'sim_annealing': value, 'sim_annealing_time': duration}
    df.loc[len(df)] = new_row_data
    df.to_csv('comparison_tables/bests.csv', index=False)

In [33]:
average = [average[0] / number_of_combinations, average[1] / number_of_combinations, average[2] / number_of_combinations]
average = [round(num, 2) for num in average]
print('average:', average)
df_avg = pd.DataFrame({'Dim': dim, 'Value': average[0], 'Time': average[1], 'Best_Iter': average[2]}, index=[0])
display(df_avg)

average: [25488.33, 1.04, 493.33]


,Dim,Value,Time,Best_Iter
0,70,25488.33,1.04,493.33


In [34]:
df = pd.read_csv('comparison_tables/averages.csv')
row_to_update = df[df['Dim'] == num_nodes]

# Check if the row exists
if not row_to_update.empty:
    # Update specific columns in the located row
    df.loc[row_to_update.index, 'sim_annealing'] = average[0]
    df.loc[row_to_update.index, 'sim_annealing_time'] = average[1]

    # Save the updated DataFrame back to the CSV file
    df.to_csv('comparison_tables/averages.csv', index=False)
else:
    df.loc[len(df)] = {'Dim': num_nodes, 'sim_annealing_perm': average[0], 'sim_annealing_time': average[1]}
    df.to_csv('comparison_tables/averages.csv', index=False)

<span style="font-size: larger;">Variable neigborhood search</span>

In [35]:
def shaking_swap(graph, solution, k):
    new_solution = deepcopy(solution)
    selected = random.sample(range(len(solution)), 2*k)
    for i in range(0,len(selected), 2):
        index1, index2 = selected[i], selected[i+1]
        new_solution[index1], new_solution[index2] = new_solution[index2], new_solution[index1]
    return new_solution    
    

In [36]:
def shaking_inverse(graph, solution, k):
    new_solution = deepcopy(solution)
    index1 = random.randint(0, len(solution))
    if(index1 + k > len(solution)):
        index1 = len(solution) - k
    index2 = index1 + k                       
    new_solution[index1:index2+1] = reversed(new_solution[index1:index2+1])  

    return new_solution

In [37]:
def vns(graph, random_solution, value, num_iters, change_func, shaking_func, local_search_func, k_min, k_max, move_prob):
    solution = deepcopy(random_solution)
    best_i = None
    for i in range(num_iters):
        for k in range(k_min, k_max):
            #print('vns: ', solution, value, i)
            new_solution = shaking_func(graph, solution, k) #diversification  

            new_value = dg.calculate_total_edge_length(graph, new_solution)

            #print('post shaking: ', new_solution, new_value, i)
            new_solution, new_value, _= local_search_func(graph, new_solution, total_edge_length, 10, change_func)

            if new_value < value or (new_value == value and random.random() < move_prob):
                if(new_value < value):
                    best_i = i

                value = new_value

                solution = deepcopy(new_solution)
                
    return solution, value, best_i

In [38]:
start = time.time()
solution, value, iter = vns(graph, random_solution, total_edge_length, 500, change_func=make_change_inverse, shaking_func=shaking_swap, local_search_func = local_search, k_min=1, k_max=3, move_prob=0.4)
end = time.time()
duration = float("{:.2f}".format(end - start))
print(solution, value, iter)
sols.append(solution)
values.append(value)
iters.append(iter)
times.append(duration)

[20, 19, 41, 32, 55, 63, 28, 50, 23, 61, 14, 6, 7, 27, 38, 43, 66, 51, 69, 2, 33, 52, 57, 65, 11, 60, 40, 35, 58, 47, 64, 59, 48, 21, 36, 25, 4, 5, 34, 13, 0, 8, 22, 26, 16, 44, 17, 46, 56, 67, 62, 24, 30, 12, 10, 39, 42, 45, 31, 53, 29, 15, 18, 9, 54, 68, 37, 3, 1, 49] 24923 462


In [39]:
start = time.time()
solution, value, iter = vns(graph, random_solution, total_edge_length, 500, change_func=make_change_inverse, shaking_func=shaking_inverse, local_search_func = local_search, k_min=3, k_max=6, move_prob=0.4)
end = time.time()
duration = float("{:.2f}".format(end - start))
print(solution, value, iter)
sols.append(solution)
values.append(value)
iters.append(iter)
times.append(duration)

[58, 28, 62, 43, 23, 50, 49, 45, 10, 42, 38, 56, 46, 39, 30, 33, 54, 26, 55, 69, 25, 19, 66, 59, 61, 63, 32, 27, 11, 51, 17, 6, 8, 52, 16, 2, 21, 60, 31, 0, 15, 4, 35, 44, 41, 65, 5, 48, 47, 13, 34, 57, 36, 29, 24, 53, 40, 68, 64, 67, 18, 12, 22, 37, 7, 9, 14, 1, 3, 20] 24658 491


In [40]:
start = time.time()
solution, value, iter = vns(graph, random_solution, total_edge_length, 500, change_func=make_change_swap, shaking_func=shaking_swap, local_search_func = local_search, k_min=1, k_max=3, move_prob=0.4)
end = time.time()
duration = float("{:.2f}".format(end - start))
print(solution, value, iter)
sols.append(solution)
values.append(value)
iters.append(iter)
times.append(duration)

[3, 1, 55, 10, 17, 18, 49, 68, 39, 37, 26, 25, 9, 29, 22, 53, 30, 31, 44, 27, 19, 42, 45, 62, 21, 15, 64, 13, 54, 8, 67, 2, 35, 59, 33, 16, 46, 32, 52, 0, 60, 24, 58, 4, 5, 48, 56, 34, 12, 47, 63, 36, 11, 40, 51, 43, 23, 69, 66, 7, 50, 6, 57, 61, 38, 20, 14, 65, 28, 41] 24421 494


In [41]:
start = time.time()
solution, value, iter = vns(graph, random_solution, total_edge_length, 500, change_func=make_change_swap, shaking_func=shaking_inverse, local_search_func = local_search, k_min=3, k_max=6, move_prob=0.4)
end = time.time()
duration = float("{:.2f}".format(end - start))
print(solution, value, iter)
sols.append(solution)
values.append(value)
iters.append(iter)
times.append(duration)

[3, 1, 14, 20, 68, 37, 9, 18, 49, 29, 53, 26, 22, 25, 17, 27, 55, 19, 64, 13, 31, 67, 10, 44, 69, 30, 33, 42, 58, 21, 51, 39, 11, 63, 52, 16, 54, 2, 59, 32, 0, 8, 66, 15, 4, 46, 24, 40, 65, 60, 35, 5, 34, 47, 12, 36, 45, 56, 23, 48, 62, 43, 38, 50, 61, 57, 6, 7, 41, 28] 23961 403


In [42]:
start = time.time()
solution, value, iter = vns(graph, random_solution, total_edge_length, 500, change_func=make_change_scramble, shaking_func=shaking_swap, local_search_func = local_search, k_min=1, k_max=3, move_prob=0.4)
end = time.time()
duration = float("{:.2f}".format(end - start))
print(solution, value, iter)
sols.append(solution)
values.append(value)
iters.append(iter)
times.append(duration)

[20, 14, 3, 15, 53, 1, 49, 68, 37, 10, 9, 55, 29, 27, 18, 17, 25, 64, 7, 13, 30, 67, 65, 36, 26, 57, 31, 69, 22, 24, 48, 12, 40, 6, 46, 21, 54, 52, 11, 51, 66, 8, 38, 60, 35, 61, 32, 2, 47, 16, 63, 0, 19, 4, 59, 41, 39, 42, 5, 50, 33, 23, 58, 43, 44, 34, 56, 28, 62, 45] 24690 490


In [43]:
start = time.time()
solution, value, iter = vns(graph, random_solution, total_edge_length, 500, change_func=make_change_scramble, shaking_func=shaking_inverse, local_search_func = local_search, k_min=3, k_max=6, move_prob=0.4)
end = time.time()
duration = float("{:.2f}".format(end - start))
print(solution, value, iter)
sols.append(solution)
values.append(value)
iters.append(iter)
times.append(duration)

[1, 10, 55, 22, 68, 7, 49, 67, 20, 19, 0, 53, 29, 12, 15, 13, 58, 66, 61, 23, 37, 64, 34, 2, 69, 11, 31, 59, 30, 33, 27, 56, 18, 21, 16, 35, 44, 8, 32, 60, 39, 52, 63, 5, 45, 47, 46, 62, 9, 51, 42, 43, 26, 24, 17, 3, 14, 36, 57, 65, 48, 25, 50, 38, 40, 6, 4, 41, 54, 28] 24965 476


In [44]:
start = time.time()
solution, value, iter = vns(graph, random_solution, total_edge_length, 500, change_func=make_change_swap, shaking_func=shaking_swap, local_search_func = local_search_with_permutation, k_min=1, k_max=3, move_prob=0.4)
end = time.time()
duration = float("{:.2f}".format(end - start))
print(solution, value, iter)
sols.append(solution)
values.append(value)
iters.append(iter)
times.append(duration)

[9, 3, 26, 37, 1, 14, 25, 24, 36, 17, 48, 50, 40, 6, 49, 7, 64, 65, 27, 13, 67, 22, 66, 30, 69, 20, 68, 42, 38, 55, 15, 51, 12, 18, 63, 53, 52, 16, 61, 60, 11, 47, 31, 8, 19, 2, 59, 29, 39, 35, 32, 56, 33, 57, 46, 0, 34, 5, 62, 44, 45, 21, 58, 43, 54, 4, 10, 23, 41, 28] 24389 484


In [45]:
start = time.time()
solution, value, iter = vns(graph, random_solution, total_edge_length, 500, change_func=make_change_swap, shaking_func=shaking_inverse, local_search_func = local_search_with_permutation, k_min=3, k_max=6, move_prob=0.4)
end = time.time()
duration = float("{:.2f}".format(end - start))
print(solution, value, iter)
sols.append(solution)
values.append(value)
iters.append(iter)
times.append(duration)

[55, 1, 28, 41, 4, 14, 23, 6, 40, 66, 10, 15, 50, 65, 67, 3, 24, 48, 0, 12, 36, 59, 13, 17, 39, 29, 30, 60, 46, 56, 42, 58, 27, 31, 51, 35, 63, 21, 16, 8, 44, 34, 5, 45, 32, 47, 38, 2, 61, 69, 43, 57, 33, 54, 62, 26, 25, 52, 9, 53, 18, 11, 37, 64, 49, 19, 68, 7, 22, 20] 24808 497


In [46]:
start = time.time()
solution, value, iter = vns(graph, random_solution, total_edge_length, 500, change_func=make_change_inverse, shaking_func=shaking_swap, local_search_func = local_search_with_permutation, k_min=1, k_max=3, move_prob=0.4)
end = time.time()
duration = float("{:.2f}".format(end - start))
print(solution, value, iter)
sols.append(solution)
values.append(value)
iters.append(iter)
times.append(duration)

[20, 62, 28, 23, 41, 43, 65, 47, 34, 57, 50, 11, 63, 5, 7, 52, 22, 48, 45, 38, 35, 44, 2, 40, 8, 36, 6, 33, 16, 32, 60, 0, 56, 59, 46, 29, 61, 69, 24, 64, 4, 51, 13, 31, 21, 42, 15, 30, 66, 54, 27, 58, 67, 39, 17, 18, 3, 26, 9, 25, 14, 19, 37, 53, 68, 12, 49, 55, 10, 1] 24704 462


In [47]:
start = time.time()
solution, value, iter = vns(graph, random_solution, total_edge_length, 500, change_func=make_change_inverse, shaking_func=shaking_inverse, local_search_func = local_search_with_permutation, k_min=3, k_max=6, move_prob=0.4)
end = time.time()
duration = float("{:.2f}".format(end - start))
print(solution, value, iter)
sols.append(solution)
values.append(value)
iters.append(iter)
times.append(duration)

[28, 62, 54, 49, 45, 33, 42, 10, 50, 15, 46, 23, 18, 4, 29, 39, 0, 68, 67, 56, 9, 1, 44, 21, 3, 5, 32, 36, 16, 53, 43, 26, 34, 58, 12, 8, 31, 24, 2, 35, 61, 60, 41, 30, 47, 57, 48, 59, 69, 64, 38, 37, 13, 52, 17, 51, 63, 11, 40, 27, 6, 7, 66, 25, 19, 14, 55, 22, 65, 20] 24290 485


In [48]:
start = time.time()
solution, value, iter = vns(graph, random_solution, total_edge_length, 500, change_func=make_change_scramble, shaking_func=shaking_swap, local_search_func = local_search_with_permutation, k_min=1, k_max=3, move_prob=0.4)
end = time.time()
duration = float("{:.2f}".format(end - start))
print(solution, value, iter)
sols.append(solution)
values.append(value)
iters.append(iter)
times.append(duration)

[55, 49, 14, 7, 25, 10, 12, 53, 54, 48, 37, 38, 61, 6, 17, 58, 26, 9, 27, 57, 66, 69, 51, 36, 46, 23, 42, 67, 33, 30, 59, 16, 45, 60, 24, 15, 56, 13, 40, 43, 4, 2, 8, 64, 34, 31, 63, 44, 18, 29, 11, 32, 47, 62, 21, 35, 0, 39, 52, 5, 68, 50, 1, 41, 22, 20, 19, 3, 65, 28] 24845 441


In [49]:
start = time.time()
solution, value, iter = vns(graph, random_solution, total_edge_length, 500, change_func=make_change_scramble, shaking_func=shaking_inverse, local_search_func = local_search_with_permutation, k_min=3, k_max=6, move_prob=0.4)
end = time.time()
duration = float("{:.2f}".format(end - start))
print(solution, value, iter)
sols.append(solution)
values.append(value)
iters.append(iter)
times.append(duration)

[28, 1, 20, 3, 7, 55, 22, 30, 29, 14, 67, 49, 9, 53, 61, 12, 57, 23, 32, 41, 46, 10, 33, 6, 36, 27, 51, 21, 45, 17, 59, 39, 44, 35, 31, 68, 64, 16, 34, 11, 63, 69, 15, 18, 8, 56, 47, 13, 60, 58, 48, 26, 42, 38, 43, 2, 24, 4, 37, 25, 40, 50, 5, 52, 66, 65, 0, 19, 54, 62] 24753 467


In [50]:
n = len(graph)

oc = False

shakings = [shaking_swap, shaking_inverse]
changes_funcs = [make_change_swap, make_change_inverse, make_change_scramble]
search_funcs = [local_search, local_search_with_permutation]

number_of_combinations = 0

sols = []
values =[]
times = []

methods = []

results = []

best = float('inf')
best_i = -1

average = [0,0]

k_min = int(0.1 * num_nodes)
k_max = int(0.3 * num_nodes)
for shake in shakings:
    for change in changes_funcs:
        for search in search_funcs:
            if shake.__name__ == 'shaking_inverse':
                k_min = int(0.3 * num_nodes)
                k_max = int(0.6 * num_nodes)

            start = time.time()    
            sol, value, iter = vns(graph, 
                                   random_solution, 
                                   total_edge_length, 
                                   500, 
                                   change_func=change, 
                                   shaking_func=shake, 
                                   local_search_func = search, 
                                   k_min=k_min,
                                   k_max=k_max, 
                                   move_prob=0.4)
            end = time.time()
            duration = float("{:.2f}".format(end - start))
            
            print(shake.__name__, change.__name__, search.__name__, ' : ', sol, value, iter, '\n')
            
            k_min = int(0.1 * num_nodes)
            k_max = int(0.3 * num_nodes)

            sols.append(sol)
            values.append(value)
            iters.append(iter)
            times.append(duration)

            methods.append(str(shake.__name__ + ', ' + change.__name__ + ', ' + search.__name__))

            results.append({'Dim': num_nodes, 'Method': methods[number_of_combinations], 'Value': abs(value), 'Time': duration})

            if value == best:
                if duration <= times[best_i]:    
                    best = value
                    best_i = number_of_combinations

            if value < best:
                best = value
                best_i = number_of_combinations            

            average[0] += value
            average[1] += duration

            number_of_combinations += 1    


shaking_swap make_change_swap local_search  :  [23, 20, 19, 1, 25, 27, 49, 55, 28, 30, 9, 69, 14, 48, 3, 37, 66, 42, 6, 17, 40, 38, 64, 59, 29, 24, 11, 22, 52, 13, 67, 31, 26, 65, 51, 8, 2, 18, 53, 44, 50, 39, 15, 63, 12, 61, 32, 68, 56, 36, 46, 0, 60, 4, 33, 45, 54, 47, 43, 5, 41, 21, 10, 16, 57, 34, 35, 58, 62, 7] 25614 294 

shaking_swap make_change_swap local_search_with_permutation  :  [53, 7, 49, 25, 45, 1, 37, 39, 3, 46, 24, 30, 5, 9, 26, 35, 12, 15, 48, 14, 36, 23, 18, 64, 40, 10, 56, 54, 43, 11, 4, 34, 57, 47, 61, 2, 31, 60, 58, 59, 42, 17, 16, 68, 8, 69, 51, 0, 41, 52, 32, 19, 21, 22, 62, 63, 65, 66, 44, 29, 6, 67, 33, 20, 13, 38, 50, 55, 27, 28] 25552 396 

shaking_swap make_change_inverse local_search  :  [56, 10, 43, 42, 57, 44, 45, 5, 58, 34, 12, 47, 39, 28, 48, 29, 26, 2, 46, 23, 24, 21, 41, 54, 59, 33, 8, 52, 62, 18, 65, 4, 7, 66, 17, 15, 60, 37, 32, 67, 49, 30, 11, 50, 13, 55, 19, 22, 63, 35, 0, 53, 27, 6, 36, 31, 61, 64, 14, 9, 38, 51, 16, 20, 69, 1, 40, 68, 25, 3] 26

In [51]:
df = pd.DataFrame(results)
display(df)
df.to_csv('comparison_tables/vns.csv', mode='a', header=not pd.io.common.file_exists('comparison_tables/vns.csv'), index=False)

,Dim,Method,Value,Time
0,70,"shaking_swap, make_change_swap, local_search",25614,84.11
1,70,"shaking_swap, make_change_swap, local_search_w...",25552,195.88
2,70,"shaking_swap, make_change_inverse, local_search",26015,80.50
3,70,"shaking_swap, make_change_inverse, local_searc...",25902,198.20
4,70,"shaking_swap, make_change_scramble, local_search",25430,82.64
5,70,"shaking_swap, make_change_scramble, local_sear...",25837,199.05
6,70,"shaking_inverse, make_change_swap, local_search",25185,120.64
7,70,"shaking_inverse, make_change_swap, local_searc...",25064,316.90
8,70,"shaking_inverse, make_change_inverse, local_se...",25562,121.26
9,70,"shaking_inverse, make_change_inverse, local_se...",25125,314.93


In [52]:
print('best:', methods[best_i], values[best_i], times[best_i])
df_best = pd.DataFrame({'Dim': num_nodes, 'Method':  methods[best_i], 'Value': values[best_i], 'Time': times[best_i]}, index=[0])
display(df_best)

best: shaking_inverse, make_change_swap, local_search_with_permutation 25064 316.9


,Dim,Method,Value,Time
0,70,"shaking_inverse, make_change_swap, local_searc...",25064,316.9


In [53]:
df = pd.read_csv('comparison_tables/bests.csv')
row_to_update = df[df['Dim'] == num_nodes]

# Check if the row exists
if not row_to_update.empty:
    # Update specific columns in the located row
    df.loc[row_to_update.index, 'vns'] = values[best_i]
    df.loc[row_to_update.index, 'vns_time'] = times[best_i]

    # Save the updated DataFrame back to the CSV file
    df.to_csv('comparison_tables/bests.csv', index=False)
else:
    new_row_data = {'Dim': num_nodes, 'vns': value, 'vns_time': duration}
    df.loc[len(df)] = new_row_data
    df.to_csv('comparison_tables/bests.csv', index=False)

In [54]:
average = [average[0] / number_of_combinations, average[1] / number_of_combinations]
average = [round(num, 2) for num in average]
print('average:', average)
df_avg = pd.DataFrame({'Dim': num_nodes, 'Value': average[0], 'Time': average[1]}, index=[0])
display(df_avg)

average: [25518.58, 180.08]


,Dim,Value,Time
0,70,25518.58,180.08


In [55]:
df = pd.read_csv('comparison_tables/averages.csv')
row_to_update = df[df['Dim'] == num_nodes]

# Check if the row exists
if not row_to_update.empty:
    # Update specific columns in the located row
    df.loc[row_to_update.index, 'vns'] = average[0]
    df.loc[row_to_update.index, 'vns_time'] = average[1]

    # Save the updated DataFrame back to the CSV file
    df.to_csv('comparison_tables/averages.csv', index=False)
else:
    df.loc[len(df)] = {'Dim': num_nodes, 'vns_perm': average[0], 'vns_time': average[1]}
    df.to_csv('comparison_tables/averages.csv', index=False)